In [311]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from itertools import combinations
import os
import sys
from collections import OrderedDict
import collections
import csv
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [312]:
# Change to True to run the program on full dataset
isProd = False

In [313]:
number_cores = 3
memory_gb = 8
# Create a configuration object and
# set the name of the application
conf = (
    SparkConf()
        .setAppName("SparkTask")
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)
# Create a Spark Context object
sc = SparkContext(conf=conf)
spr = SparkSession.builder.appName("naruskevica").getOrCreate()

# Solution

In [314]:
# Read the input file
if isProd:
    if not os.path.exists('input/Reviews.csv'):
        sc.stop()
        raise Exception("""
            Download the 'Reviews.csv' file from https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
            and put it in 'input' folder
        """)
    else:
        inputRdd = sc.textFile("input/Reviews.csv")
else:
    inputRdd = sc.textFile("input/Sample.csv")
    inputCsv = spr.read.csv(
        path="input/Sample.csv",
        sep=",",
        header=True,
        quote='"',
        inferSchema=True,
        )

In [315]:
# Remove the header
filteredInput = inputRdd.filter(lambda line: line.startswith("Id,") == False)

# Note

Make sure you save program output to output folder

Create tuple where first elemnt of tupele is userId and secode is list of ProductId

In [316]:
my_map = {}
for i in filteredInput.collect():
    st = i.split(",")
    if my_map.get(st[2]) == None:
        my_map[st[2]] = [st[1]]
    else:
        new_list = my_map.get(st[2])
        new_list.append(st[1])
        my_map[st[2]] = new_list
answer_list = [(i, my_map.get(i)) for i in my_map]

Create all combination from list of purchase for every user

In [317]:
list_of_combination = []
for k, v in answer_list:
    for comb in combinations(v, 2):
        list_of_combination.append((",".join(comb), 1))

Сount all combination of purchase and sort it and convert to list

In [318]:
def count_pair(list_of_comb):
    dict_of_pair = {}
    for k, v in list_of_comb:
        if dict_of_pair.get(k) == None:
            dict_of_pair[k] = 1
        else:
            count = dict_of_pair.get(k) + 1
            dict_of_pair[k] = count
    return dict_of_pair
    
dict_of_pair = count_pair(list_of_combination)
sorted_list_pair = sorted(dict_of_pair.items(), key=lambda item: item[1], reverse=True)

[('B1,B5', 4), ('B3,B5', 4), ('B1,B3', 3), ('B4,B5', 2), ('B1,B1', 1), ('B3,B1', 1), ('B5,B1', 1), ('B1,B4', 1), ('B3,B4', 1), ('B5,B5', 1)]


Save final result to result.csv file

In [319]:
with open("output/result.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(sorted_list_pair[0:10])

# Stop the Spark Context

In [320]:
sc.stop()